## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
# city_data_df = pd.read_csv("weather_database/WeatherPy_database.csv")
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gasa,PS,31.5000,34.4667,76.98,67,0,7.94,clear sky
1,1,Bulungu,CD,-4.5500,18.6000,70.21,90,100,1.59,light rain
2,2,Wakkanai,JP,45.4094,141.6739,67.98,68,40,11.50,scattered clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,41.11,81,0,12.66,clear sky
4,4,Puerto Ayora,EC,-0.7393,-90.3518,69.22,90,87,6.91,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 75
What is your desired maximum temperature for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gasa,PS,31.5000,34.4667,76.98,67,0,7.94,clear sky
6,6,Salalah,OM,17.0151,54.0924,80.69,89,20,1.14,few clouds
14,14,Avarua,CK,-21.2078,-159.7750,78.85,69,20,3.44,few clouds
15,15,Butaritari,KI,3.0707,172.7902,83.71,73,91,13.91,light rain
17,17,Tessalit,ML,20.1986,1.0114,84.72,16,1,8.14,clear sky
...,...,...,...,...,...,...,...,...,...,...
675,675,Pitimbu,BR,-7.4706,-34.8086,75.79,84,23,17.96,few clouds
676,676,Joetsu,JP,37.1483,138.2364,81.72,59,32,7.36,scattered clouds
678,678,Tongling,CN,30.9500,117.7833,78.87,83,100,7.67,overcast clouds
699,699,Pedasi,PA,7.5333,-80.0333,79.07,84,99,4.90,overcast clouds


##### 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gasa,PS,31.5000,34.4667,76.98,67,0,7.94,clear sky
6,6,Salalah,OM,17.0151,54.0924,80.69,89,20,1.14,few clouds
14,14,Avarua,CK,-21.2078,-159.7750,78.85,69,20,3.44,few clouds
15,15,Butaritari,KI,3.0707,172.7902,83.71,73,91,13.91,light rain
17,17,Tessalit,ML,20.1986,1.0114,84.72,16,1,8.14,clear sky
...,...,...,...,...,...,...,...,...,...,...
675,675,Pitimbu,BR,-7.4706,-34.8086,75.79,84,23,17.96,few clouds
676,676,Joetsu,JP,37.1483,138.2364,81.72,59,32,7.36,scattered clouds
678,678,Tongling,CN,30.9500,117.7833,78.87,83,100,7.67,overcast clouds
699,699,Pedasi,PA,7.5333,-80.0333,79.07,84,99,4.90,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Gasa,PS,76.98,clear sky,31.5000,34.4667,
6,Salalah,OM,80.69,few clouds,17.0151,54.0924,
14,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,
15,Butaritari,KI,83.71,light rain,3.0707,172.7902,
17,Tessalit,ML,84.72,clear sky,20.1986,1.0114,
25,Sao Filipe,CV,78.40,broken clouds,14.8961,-24.4956,
37,Kahului,US,83.46,scattered clouds,20.8947,-156.4700,
38,Murray Bridge,AU,75.49,broken clouds,-35.1167,139.2667,
42,Bengkulu,ID,82.85,broken clouds,-3.8004,102.2655,
45,Hilo,US,80.33,moderate rain,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
clean_hotel = []
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
      
    except (IndexError):
        print("Cannot find hotel...skipping.")   

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()
hotel_df.isnull().count()

City                   187
Country                187
Max Temp               187
Current Description    187
Lat                    187
Lng                    187
Hotel Name             187
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()
hotel_df.isnull().count()

City                   187
Country                187
Max Temp               187
Current Description    187
Lat                    187
Lng                    187
Hotel Name             187
dtype: int64

In [12]:

hotel_df.count()

City                   187
Country                187
Max Temp               187
Current Description    187
Lat                    187
Lng                    187
Hotel Name             187
dtype: int64

In [13]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [21]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map.
locations = ()
hotel_info = ()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))